# Import File

In [1]:
import pandas as pd
import numpy as np

In [2]:
import sys
sys.path.append('../lending_club')
import config
from data_prep import get_lending_club_data, refine_features

In [ ]:
%%time
approved_loans = get_lending_club_data(config.APPROVED_LOANS_CSV, 
                                       clean_file=True)
approved_loans_df = approved_loans.compute()

In [ ]:
%%time
approved_loans_df = refine_features(approved_loans_df)

In [3]:
%%time
approved = pd.read_pickle('../data/approved.pickle')

Wall time: 2.98 s


In [ ]:
approved.dtypes

### Removed NA's

In [ ]:
approved.drop(['emp_title', 'earliest_cr_line', 'grade', 'zip_code', 'last_pymnt_d'], axis = 1, inplace = True)

### Dummification

In [ ]:
%%time
for d in ['addr_state', 'application_type', 'disbursement_method', 'home_ownership', \
          'initial_list_status', 'purpose', 'sub_grade','verification_status']:
    temp = pd.get_dummies(approved[d], prefix=d, prefix_sep='__')
    temp = temp.drop(d+'__'+str(approved[d].mode()[0]), axis=1)
    #drop the most common one (mode), making it the base case
    approved = pd.concat([approved.drop(d, axis=1),temp], axis=1)
    #merge back with main df

### Get 5 year loan DataFrame


In [ ]:
from model_prep import divide_by_term

five_year = divide_by_term(approved, 60)

In [ ]:
five_year.drop('issue_d', axis = 1, inplace = True)

### Linear 

In [ ]:
five_year.shape

In [ ]:
target = five_year['loan_status']
five_year_features = five_year.drop('loan_status', axis = 1)

In [ ]:
five_year['loan_status'].value_counts()

In [ ]:
from model_prep import split_data

Xtrain,Xtest,ytrain,ytest = split_data(five_year, test_size = 0.3)

In [ ]:
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import confusion_matrix

In [ ]:
%%time
svm = LinearSVC()
svm.set_params(max_iter = 1e8)

In [ ]:
%%time
svm.fit(Xtrain, ytrain)

In [ ]:
%%time
svm.score(Xtrain, ytrain)

In [ ]:
svm.score(Xtest, ytest)

In [ ]:
%%time
confusion_matrix(ytest, svm.predict(Xtest))

In [ ]:
from joblib import dump, load

In [ ]:
dump(svm, 'svm.model')

In [ ]:
# For loading model back in
svm = load('svm.model')

### Linear GridSearchCV

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
%%time
param_dict = {'C':np.linspace(8500, 8900, 10)}
grid = GridSearchCV(svm, param_dict, return_train_score = True, n_jobs = -1)
grid.fit(Xtrain, ytrain)

In [ ]:
%%time
grid.best_params_

In [ ]:
%%time
grid.best_score_

In [ ]:
%%time
svc_best = grid.best_estimator_

In [ ]:
%%time
svc_best.score(Xtest, ytest)

In [ ]:
%%time
confusion_matrix(ytest, svc_best.predict(Xtest))

In [ ]:
dump(svc_best, 'svc_best.model')

In [ ]:
svc_best = load('svc_best.model')

### Polynomial

In [ ]:
%%time
svmPoly = SVC(kernel = 'poly', max_iter = 1e8)

In [ ]:
%%time
svmPoly.fit(Xtrain, ytrain)

In [ ]:
%%time
svmPoly.score(Xtrain, ytrain)

In [ ]:
svmPoly.score(Xtest, ytest)

In [ ]:
%%time
confusion_matrix(ytest, svmPoly.predict(Xtest))

### Polynomial GridSearchCV

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
%%time
param_dict = {'C':np.linspace(1700, 1800, 10)}
gridPoly = GridSearchCV(svmPoly, param_dict, return_train_score = True, n_jobs = -1)
gridPoly.fit(Xtrain, ytrain)

In [ ]:
%%time
gridPoly.best_params_

In [ ]:
%%time
gridPoly.best_score_

In [ ]:
%%time
Poly_best = gridPoly.best_estimator_

In [ ]:
%%time
Poly_best.score(Xtest, ytest)

In [ ]:
%%time
confusion_matrix(ytest, Poly_best.predict(Xtest))

In [ ]:
dump(Poly_best, 'poly_best.model')

### Radial

In [ ]:
%%time
svmRbf = SVC(kernel = 'rbf', max_iter = 1e8)

In [ ]:
%%time
svmRbf.fit(Xtrain, ytrain)

In [ ]:
%%time
svmRbf.score(Xtrain, ytrain)

In [ ]:
%%time
confusion_matrix(ytest, svmRbf.predict(Xtest))

### Radial GridSearchCV

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
%%time
param_dict = {'C':np.linspace(0.001,0.1,5), 'gamma':np.linspace(0.0001,0.1, 10)}
gridRbf = GridSearchCV(svmRbf, param_dict, return_train_score = True, n_jobs = -1)
gridRbf.fit(Xtrain, ytrain)

In [ ]:
%%time
gridRbf.best_params_

In [ ]:
%%time
gridRbf.best_score_

In [ ]:
%%time
Rbf_best = gridRbf.best_estimator_

In [ ]:
%%time
confusion_matrix(ytest, Rbf_best.predict(Xtest))

In [ ]:
dump(Rbf_best, 'rbf_best.model')